In [14]:
import pandas as pd

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import BaselineOnly, SVDpp, SVD

In [2]:
df_col = pd.read_csv('./data/review_all_clean.csv')
df_col.head()

,rating,user_id,movie_id,reviews
0,5,A2VHSG6TZHU1OB,0001527665,Having lived in West New Guinea (Papua) during...
1,5,A1KM9FNEJ8Q171,0001527665,"More than anything, I've been challenged to fi..."
2,4,A38LY2SSHVHRYB,0001527665,This is a great movie for a missionary going i...
3,5,AHTYUW2H1276L,0001527665,This movie was in ENGLISH....it was a great su...
4,5,A3M3HCZLXW0YLF,0001527665,"This is a fascinating true story, well acted b..."


In [3]:
df_col.drop(columns='reviews', axis=1, inplace=True)
df_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3013831 entries, 0 to 3013830
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   rating    int64 
 1   user_id   object
 2   movie_id  object
dtypes: int64(1), object(2)
memory usage: 69.0+ MB


In [4]:
#df['user_id'] = df['user_id'].astype('category').cat.codes
#df['movie_id'] = df['movie_id'].astype('category').cat.codes
#num_users = df['user_id'].nunique()
#num_movies = df['movie_id'].nunique()

In [5]:
df = df_col[df_col['user_id'].isin(df_col['user_id'].value_counts()[df_col['user_id'].value_counts() >= 3].index)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1274639 entries, 7 to 3013830
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   rating    1274639 non-null  int64 
 1   user_id   1274639 non-null  object
 2   movie_id  1274639 non-null  object
dtypes: int64(1), object(2)
memory usage: 38.9+ MB


In [7]:
reader = Reader()
data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)

In [8]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [9]:
base = BaselineOnly()
base.fit(trainset)
pred = base.test(testset)
basic = accuracy.rmse(pred)

Estimating biases using als...
RMSE: 1.0554


In [10]:
print(basic)

1.0553614688118766


In [18]:
SVD_base = SVD()
results = cross_validate(SVD_base, data, measures=['RMSE'], n_jobs=-1, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0425  1.0404  1.0421  1.0444  1.0411  1.0421  0.0014  
Fit time          6.11    6.19    6.26    6.15    5.98    6.14    0.09    
Test time         2.20    2.15    1.98    1.83    1.74    1.98    0.18    


In [20]:
SVD_base = SVD()
SVD_base.fit(trainset)
predictions = SVD_base.test(testset)
kn_first = accuracy.rmse(predictions)

RMSE: 1.0406


In [19]:
SVDpp_base = SVDpp()
results = cross_validate(SVDpp_base, data, measures=['RMSE'], n_jobs=-1, verbose=True)

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0407  1.0387  1.0383  1.0364  1.0383  1.0385  0.0014  
Fit time          11.35   11.30   11.50   11.23   11.31   11.34   0.09    
Test time         4.10    4.11    4.09    4.11    3.99    4.08    0.05    


In [21]:
SVDpp_base = SVDpp()
SVDpp_base.fit(trainset)
predictions = SVDpp_base.test(testset)
kn_first = accuracy.rmse(predictions)

RMSE: 1.0375
